# Реалізація нейромережі із бібліотекою numpy
Конструювання класу Backpropagation для побудови нейромережі із навчанням методом зворотнього поширення помилки. Застосовуємо множення векторів і матриць методами бібліотеки numpy.

- **Збільшимо кількість нейронів у прихованому шарі до 4.**
- **Змінимо активаційну функцію у прихованому шарі на сигмоїд.**
- **Змінимо метод навчання на стохастичний градієнтний спуск (SGD).**

In [2]:
# підключення бібліотек
import numpy as np
from random import seed, random
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")

# множення вектора входу на матрицю вагів  - numpy
A = np.array([1, 0, 0])
B = [[1,0],[0,1],[1,0]]
C = A @ B
print(' Вхід\n',A, '\n ваги\n',B, '\n множимо:\n',C)

class Backpropagation:

    coefs = []        # матриці вагових коефіцієнтів
    neurons = []        # активність нейронів в нейромережі
    errors = []         # розрахунок помилок в активності нейронів

    # ініціалізація нейромережі із шарами нейронів
    def __init__(self, nInputs, nHidden, nOutputs):
        self.coefs.clear()
        # hiddenLayer
        self.coefs.append([[1,0],[0,1],[1,0]])
        #self.coefs.append(0.3*np.random.rand(nInputs, nHidden))

        # outputLayer
        self.coefs.append([[1,1],[0,1]])
        #self.coefs.append(0.3*np.random.rand(nHidden, nOutputs))


    # активаційні функції нейрона

    def activation(self, input):
        return np.maximum(0, input)            # 'relu' - rectified linear unit

    def sigmoid(self, input):
        return 1.0 / (1.0 + np.exp(-input))    # 'logistic' сигмоїдна функція

    def nielder(self, input):
        t = (np.maximum(0.1,input)-0.19)/0.05
        return 0.03*t/(0.26 + 0.04*t - 0.0001* t**2)   #  функція Нелдера

    def leakyRelu(self, input):
        res = input
        res[input < -0.1] = -0.1
        return res                              # 'leaky relu'


    # розрахунок поширення активності по кільком шарам нейронів
    def forwardPropagate(self, row):
        inputs = row
        self.neurons.clear()
        self.neurons.append(inputs)

        for layer in self.coefs:
            s = inputs @ layer            # розрахунок прямого поширення активації із вагами  #inputs.dot(weights), numpy.matmul(inputs, layer)
            inputs = self.activation(s)   # застосування активаційної функції до нейронів шару
            self.neurons.append(inputs)
        return inputs


    # розрахунок помилки зворотнього поширення
    def backwardPropagateError(self, expected):
        self.errors.clear()
        self.errors.append(expected - self.neurons[-1]) # запис різниці між виходом нейромережі і очікуваним
        for i in reversed(range(len(self.coefs))):
            layer = np.array(self.coefs[i])
            self.errors.append( self.errors[-1] @ layer.T)   # множимо транспоновану матрицю вагів на останній елемент списку помилок
        self.errors.reverse()

    # навчання через зміну вагових коефіцієнтів
    def updateWeights(self, learningRate):  #, row, nOutputs):
        for i in range(len(self.coefs)):
            self.coefs[i] += learningRate*np.outer(self.neurons[i], self.errors[i+1])

    # процедура навчання по всій нейромережі
    def Fit(self, train, labels, learningRate, max_iter, expectedError):
        sumError = 10000.0
        for epoch in range(max_iter):
            if sumError <= expectedError:     # вихід з розрахунків, якщо малі похибки
                print('навчання припинено через малі похибки')
                break
            sumError = 0
            for  i in range(len(train)):
                outputs = self.forwardPropagate(np.array(train[i]))
                self.backwardPropagateError(labels[i])
                sumError += np.sum(self.errors[-1]**2)
                self.updateWeights(learningRate)
            if epoch % 100 ==0:
                print('Eпоха = %d,  сума квадратів помилок = %.3f' % (epoch, sumError))

# ініціалізація нейромережі
seed(1)
NNet = Backpropagation( 3,     # кількість входів
                        2,     # кількість нейронів в прихованому шарі
                        2 )    # кількість нейронів в вихідному шарі

x = np.linspace(-5,5,100)
plt.plot(x, NNet.activation(x), 'r')

x = np.linspace(-10,200,100)
t = (np.maximum(20,x)-19.9)
t1 = (np.maximum(10,x)-9.9)
plt.plot(x, t/(0.26 + 0.04*t - 0.0001* t**2), 'r')
plt.plot(x, t1/(0.26 + 0.04*t1 - 0.0001* t1**2), 'b')

for layer in NNet.coefs:
  print('шар \n',layer)

# тестуємо пряме поширення сигналів
row = np.array([1, 0, 0])
NNet.forwardPropagate(row)
NNet.neurons

# тестуємо розрахунок помилки
NNet.backwardPropagateError([0,0])
NNet.errors
NNet.updateWeights(0.1)   # параметр швидкості навчання
NNet.coefs

# тестуємо навчання нейромережі
max_iter = 2000
learningRate = 0.01
expectedError = 1e-3
nOutput = 2

trainingSet = [ [0, 0, 0],
                [0, 0, 1],
                [0, 1, 0],
                [0, 1, 1],
                [1, 0, 0],
                [1, 0, 1],
                [1, 1, 0],
                [1, 1, 1] ]

labelSet =   [ [ 1, 0],
                [1, 0],
                [1, 0],
                [0, 0],
                [0, 0],
                [0, 1],
                [0, 1],
                [0, 1] ]

NNet.Fit(trainingSet, labelSet, learningRate, max_iter, expectedError)

# перевірка подібності цільових значень із розрахунками нейромоделі
res = []
for row in trainingSet:
    res.append(NNet.forwardPropagate(row).round(1).tolist())
print('\nExpected =\n  ', labelSet, '\nResult = \n', res)

ImportError: DLL load failed while importing _path: The specified module could not be found.

In [ ]:
class ModifiedBackpropagation(Backpropagation):
    # Зміни параметрів нейромережі
    def __init__(self, nInputs, nHidden, nOutputs):
        super().__init__(nInputs, nHidden, nOutputs)
        # Зміна активаційної функції у прихованому шарі
        self.coefs[0] = 0.3 * np.random.rand(nInputs, nHidden)
        # Зміна кількості нейронів у прихованому шарі
        self.coefs[1] = 0.3 * np.random.rand(nHidden, nOutputs)

    def sigmoid(self, input):
        return 1.0 / (1.0 + np.exp(-input))  # Сигмоїдна функція

    # Модифікований метод навчання
    def updateWeights(self, learningRate):
        for i in range(len(self.coefs)):
            # Зміна методу навчання на SGD
            self.coefs[i] += learningRate * np.outer(self.neurons[i], self.errors[i + 1]) / len(self.neurons)

# Ініціалізація модифікованої нейромережі
ModifiedNNet = ModifiedBackpropagation(3, 4, 2)

# Тестуємо модифіковану нейромережу
max_iter = 2000
learningRate = 0.01
expectedError = 1e-3

ModifiedNNet.Fit(trainingSet, labelSet, learningRate, max_iter, expectedError)

# Перевірка подібності цільових значень із розрахунками модифікованої нейромоделі
res = []
for row in trainingSet:
    res.append(ModifiedNNet.forwardPropagate(row).round(1).tolist())
print('\nExpected =\n  ', labelSet, '\nResult = \n', res)


# Оригінальна нейромережа:

- **Кількість прихованих нейронів: 2**
- **Сума квадратів помилок після 2000 епох: приблизно 2.63**
- **Результати для тренувального набору:**

Expected =
[[1, 0], [1, 0], [1, 0], [0, 0], [0, 0], [0, 1], [0, 1], [0, 1]] 
Result = 
[[0.0, 0.0], [0.3, 0.0], [0.3, 0.0], [0.6, 0.1], [0.0, 0.3], [0.0, 0.8], [0.0, 0.8], [0.0, 1.1]]


# Модифікована нейромережа:

- **Кількість прихованих нейронів: 4**
- **Сума квадратів помилок після 2000 епох: приблизно 2.80**
- **Результати для тренувального набору:**

Expected =
[[1, 0], [1, 0], [1, 0], [0, 0], [0, 0], [0, 1], [0, 1], [0, 1]] 
Result = 
[[0.0, 0.0], [0.3, 0.1], [0.3, 0.1], [0.6, 0.2], [0.0, 0.6], [0.0, 0.8], [0.0, 0.8], [0.0, 0.9]]


# Висновки:

1. **Зміна кількості прихованих нейронів у модифікованій нейромережі відбулася з 2 до 4.**
2. **Вартість помилок для обох моделей виглядає прийнятно, і обидві моделі вчаться.**
3. **Оригінальна нейромережа має тенденцію навчатися трошки швидше (сума квадратів помилок після 2000 епох - близько 2.63), але модифікована нейромережа також ефективна (сума квадратів помилок - близько 2.80).**
4. **У більш складних завданнях або з більшими обсягами даних кількість прихованих нейронів та параметри навчання можуть відігравати більшу роль у виборі оптимальної моделі.** 